In [1]:
import pandas as pd
import fuzzywuzzy

In [2]:
df = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.csv')

In [3]:
df.head()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


1. Renaming columns 

In [4]:
df = df.rename(columns={
    'Timestamp': 'date',
    'How old are you?': 'age',
    'What industry do you work in?': 'industry',
    'Job title': 'job_title',
    'If your job title needs additional context, please clarify here:': 'job_title_additions',
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": 'annual_salary',
    'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.': 'additional_monetary',
    'Please indicate the currency': 'currency',
    'If "Other," please indicate the currency here: ': 'other_currency',
    'If your income needs additional context, please provide it here:': 'additional_income_context',
    'What country do you work in?': 'job_country',
    "If you're in the U.S., what state do you work in?": 'state_us',
    'What city do you work in?': 'job_city',
    'How many years of professional work experience do you have overall?': 'job_experience',
    'How many years of professional work experience do you have in your field?': 'field_experience',
    'What is your highest level of education completed?': 'education',
    'What is your gender?': 'gender',
    'What is your race? (Choose all that apply.)': 'race'
})

In [5]:
df.head()

,date,age,industry,job_title,job_title_additions,annual_salary,additional_monetary,currency,other_currency,additional_income_context,job_country,state_us,job_city,job_experience,field_experience,education,gender,race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


2. Data types

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27839 entries, 0 to 27838
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date                       27839 non-null  object 
 1   age                        27839 non-null  object 
 2   industry                   27769 non-null  object 
 3   job_title                  27839 non-null  object 
 4   job_title_additions        7201 non-null   object 
 5   annual_salary              27839 non-null  object 
 6   additional_monetary        20630 non-null  float64
 7   currency                   27839 non-null  object 
 8   other_currency             189 non-null    object 
 9   additional_income_context  3025 non-null   object 
 10  job_country                27839 non-null  object 
 11  state_us                   22888 non-null  object 
 12  job_city                   27764 non-null  object 
 13  job_experience             27839 non-null  obj

In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df['age'] = df['age'].astype('category')
df['annual_salary'] = pd.to_numeric(df['annual_salary'].str.replace(',', ''))

---
# Cleaning currency

In [9]:
df['currency'].value_counts()

USD        23208
CAD         1659
GBP         1578
EUR          632
AUD/NZD      499
Other        148
CHF           37
SEK           37
JPY           23
ZAR           14
HKD            4
Name: currency, dtype: int64

In [10]:
df['other_currency'].value_counts()

USD                         11
NOK                         10
INR                          9
MYR                          8
DKK                          8
                            ..
Taiwanese dollars            1
RMB (chinese yuan)           1
-                            1
NIS (new Israeli shekel)     1
Singapore Dollara            1
Name: other_currency, Length: 113, dtype: int64

Why USD in other_currency. Let's check

In [11]:
df[df['other_currency'] == 'USD'].head()

,date,age,industry,job_title,job_title_additions,annual_salary,additional_monetary,currency,other_currency,additional_income_context,job_country,state_us,job_city,job_experience,field_experience,education,gender,race
2473,2021-04-27 11:44:54,25-34,Nonprofits,Special Events Manager,NaN,48068,500.0,Other,USD,NaN,USA,Maine,Portland,8 - 10 years,8 - 10 years,College degree,Woman,White
10150,2021-04-27 20:00:21,45-54,Property or Construction,Staff Accountant,NaN,49000,2500.0,Other,USD,NaN,USA,Illinois,Rockford,31 - 40 years,21 - 30 years,Some college,Woman,White
11434,2021-04-28 01:21:06,25-34,Law,L&D Specialist,NaN,91000,NaN,Other,USD,NaN,USA,District of Columbia,"Washington, DC",5-7 years,5-7 years,"Professional degree (MD, JD, etc.)",Woman,Black or African American
11788,2021-04-28 05:16:07,45-54,Health care,RN,NaN,100000,20000.0,Other,USD,NaN,US,Oregon,Portland,11 - 20 years,5-7 years,College degree,Woman,White
12481,2021-04-28 09:57:04,45-54,"Accounting, Banking & Finance",Director of Human Resources,NaN,94000,3000.0,USD,USD,Bonus,United States,Texas,Waco,11 - 20 years,11 - 20 years,College degree,Woman,White


In [12]:
currency = set(df['currency'])

In [13]:
currency

{'AUD/NZD',
 'CAD',
 'CHF',
 'EUR',
 'GBP',
 'HKD',
 'JPY',
 'Other',
 'SEK',
 'USD',
 'ZAR'}

In [14]:
# define function which finds main currencies in 'other_currency' and writes its value in 'currency'
def move_currency(main_currencies):
    for curr in main_currencies:
        df.loc[
            df['other_currency'].str.lower().str.contains(curr.lower(), na=False), ['currency', 'other_currency']
        ] = [curr, None]

In [15]:
move_currency(currency)

In [16]:
df.loc[
    df['other_currency'].str.upper().str.contains('AUD', na=False) | 
    df['other_currency'].str.upper().str.contains('NZD', na=False) 
    , ['currency', 'other_currency']] = ['AUD/NZD', None]


In [17]:
#pd.set_option("display.max_rows", 70)

In [18]:
other_curr = df['other_currency'].value_counts()
other_curr

NOK                       10
INR                        9
MYR                        8
DKK                        8
SGD                        7
                          ..
55,000                     1
ILS/NIS                    1
Norwegian kroner (NOK)     1
TRY                        1
Singapore Dollara          1
Name: other_currency, Length: 99, dtype: int64

In [19]:
popular_other_curr = set(other_curr[other_curr > 1].index.str.upper())
# for index, value in other_curr.items():
#     print(index, value)
popular_other_curr

{'ARS',
 'BRL',
 'CNY',
 'CZK',
 'DKK',
 'ILS',
 'INR',
 'KRW',
 'MXN',
 'MYR',
 'NOK',
 'NONE',
 'NTD',
 'PLN',
 'SGD',
 'THB'}

In [20]:
move_currency(popular_other_curr)

In [21]:
df['other_currency'].value_counts()

Peso Argentino                                                                                                      1
Rs                                                                                                                  1
Philippine peso (PHP)                                                                                               1
PHP                                                                                                                 1
Many non-salary benefits - travel, free healthcare for self, very low for family, non-taxable housing allowance     1
                                                                                                                   ..
-                                                                                                                   1
RMB (chinese yuan)                                                                                                  1
Philippine Peso                                         

In [22]:
move_currency(['PHP'])

In [23]:
df.loc[(~df['other_currency'].isna()) & (df['currency'] != 'Other'), 'other_currency'] = None

In [24]:
df['other_currency'].value_counts()

Peso Argentino              1
Equity                      1
Rupees                      1
croatian kuna               1
Thai Baht                   1
THAI  BAHT                  1
Mexican Pesos               1
Korean Won                  1
Danish Kroner               1
Philippine Pesos            1
Argentine Peso              1
IDR                         1
RM                          1
SAR                         1
Israeli Shekels             1
Polish Złoty                1
BR$                         1
LKR                         1
China RMB                   1
IDR                         1
Philippine Peso             1
RMB (chinese yuan)          1
NIS (new Israeli shekel)    1
US Dollar                   1
TRY                         1
czech crowns                1
Bdt                         1
Mexican pesos               1
Indian rupees               1
TTD                         1
Singapore Dollara           1
Name: other_currency, dtype: int64

In [25]:
curr_relate = {
    'ARS': ['Peso Argentino', 'Argentine Peso'],
    'INR': ['Rupees'],
    'HRK': ['croatian kuna'],
    'THB': ['Thai Baht', 'THAI  BAHT'],
    'MXN': ['Mexican Pesos'],
    'KRW': ['Korean Won'],
    'DKK': ['Danish Kroner'],
    'PHP': ['Philippine Pesos', 'Philippine Peso'],
    'IDR': ['IDR'],
    'RMB': ['RMB'],
    'ILS': ['NIS', 'Israeli Shekels'],
    'USD': ['US Dollar'],
    'TRY': ['TRY'],
    'CZK': ['czech crowns'],
    'BDT': ['Bdt'],
    'TTD': ['TTD'],
    'SGD': ['Singapore Dollara'],
    'LKR': ['LKR'],
    'SAR': ['SAR'],
    'PLN': ['Polish Złoty'],
    'BRL': ['BR'],
    'MYR': ['RM']
}

In [26]:
for key in curr_relate:
    for name in curr_relate[key]:
        cond = df['other_currency'].str.lower().str.contains(name.lower(), na=False)
        df.loc[cond, ['currency', 'other_currency']] = [key, None]

In [27]:
df['other_currency'].value_counts()

Equity    1
Name: other_currency, dtype: int64

In [28]:
df[~df['other_currency'].isna()]

,date,age,industry,job_title,job_title_additions,annual_salary,additional_monetary,currency,other_currency,additional_income_context,job_country,state_us,job_city,job_experience,field_experience,education,gender,race
14915,2021-04-28 15:43:39,25-34,"Marketing, Advertising & PR",Marketing Content Creator,NaN,65000,5000.0,Other,Equity,NaN,United States,New York,Brooklyn,8 - 10 years,2 - 4 years,College degree,Woman,White


In [29]:
df['currency'].value_counts().sort_index()

ARS            8
AUD/NZD      504
BDT            1
BRL            8
CAD         1660
CHF           38
CNY            2
CZK            6
DKK           13
EUR          633
GBP         1579
HKD            4
HRK            1
IDR            2
ILS            7
INR           12
JPY           23
KRW            4
LKR            1
MXN            4
MYR            9
NOK           12
NONE           2
NTD            2
Other          6
PHP            6
PLN            6
RMB            2
SAR            1
SEK           38
SGD           10
THB            4
TRY            1
TTD            1
USD        23214
ZAR           15
Name: currency, dtype: int64

In [30]:
df['other_currency'].value_counts()

Equity    1
Name: other_currency, dtype: int64

In [31]:
df['currency'].value_counts()

USD        23214
CAD         1660
GBP         1579
EUR          633
AUD/NZD      504
SEK           38
CHF           38
JPY           23
ZAR           15
DKK           13
NOK           12
INR           12
SGD           10
MYR            9
BRL            8
ARS            8
ILS            7
CZK            6
PHP            6
PLN            6
Other          6
HKD            4
THB            4
MXN            4
KRW            4
NTD            2
RMB            2
CNY            2
NONE           2
IDR            2
SAR            1
BDT            1
LKR            1
TRY            1
TTD            1
HRK            1
Name: currency, dtype: int64

In [32]:
df['currency'] = df['currency'].astype('category')

## finish cleaning currency
---

# Cleaning country

In [33]:
df['job_country'].value_counts()

United States                                            8929
USA                                                      7897
US                                                       2592
Canada                                                   1557
United States                                             659
                                                         ... 
Czech Republic                                              1
Bangladesh                                                  1
Eritrea                                                     1
For the United States government, but posted overseas       1
Bosnia and Herzegovina                                      1
Name: job_country, Length: 366, dtype: int64

---
366 different countries while there are around 200 in the world

In [34]:
# download dictionary of countries names
import datapackage

data_url = 'https://datahub.io/core/country-list/datapackage.json'

try:
    # to load Data Package into storage
    package = datapackage.Package(data_url)
    # to load only tabular data
    resources = package.resources
    for resource in resources:
        if resource.tabular:
            dict_country = pd.read_csv(resource.descriptor['path'])
    print(dict_country)
except Exception:
    print('dictionary of countries was not download')


                  Name Code
0          Afghanistan   AF
1        Åland Islands   AX
2              Albania   AL
3              Algeria   DZ
4       American Samoa   AS
..                 ...  ...
244  Wallis and Futuna   WF
245     Western Sahara   EH
246              Yemen   YE
247             Zambia   ZM
248           Zimbabwe   ZW

[249 rows x 2 columns]


In [ ]:
# test on small data
for ind, country_name in dict_country[:50]['Name'].items():
    result = fuzzywuzzy.process.extract(country_name, df['job_country'], limit=15000)
    df[:10].loc[df['job_country'].isin([i[0] for i in result if i[1] >= 80]),'new_country'] = country_name
    

In [ ]:
# full data changer
for ind, country_name in dict_country['Name'].items():
    result = fuzzywuzzy.process.extract(country_name, df['job_country'], limit=20000)
    df.loc[df['job_country'].isin([i[0] for i in result if 100 > i[1] > 90 ]),'new_country'] = country_name

In [ ]:
df

3. Checking duplicates and null values

In [ ]:
print(df.duplicated().sum())
print(df.isna().sum())